In [1]:
from models.dpa_p2pnet import build_model
import glob
from mmengine.config import Config

cfg = Config.fromfile('config/cryoPoint.py')

model = build_model(cfg)
model_without_ddp = model
# 检查 model 和 model_without_ddp 是否指向同一个对象
if model is model_without_ddp:
    print("model 和 model_without_ddp 指向同一个对象")
else:
    print("model 和 model_without_ddp 指向不同的对象")




model 和 model_without_ddp 指向同一个对象


In [2]:
  # 冻结模型的所有参数
for params in model_without_ddp.parameters():
    params.requires_grad = False

In [3]:
for name, param in model.named_parameters():

    if param.requires_grad:
        print(f"Parameter '{name}' is trainable.")
    else:
        # print(f"Parameter '{name}' is frozen.")
        continue
      # 没输出说明冻结所有参数

In [25]:
denoise_params = []
for params in model_without_ddp.backbone.unet_encoder.parameters():
    params.requires_grad = True
    denoise_params += [params]
for params in model_without_ddp.backbone.unet_decoder.parameters():
    params.requires_grad = True
    denoise_params += [params]
for params in model_without_ddp.backbone.feature_selection.parameters():
    params.requires_grad = True
    denoise_params += [params]

seg_params = []
for params in model_without_ddp.backbone.backbone.parameters():
    params.requires_grad = True
    seg_params += [params]
for params in model_without_ddp.backbone.neck.parameters():
    params.requires_grad = True
    seg_params += [params]
for params in model_without_ddp.backbone.neck1.parameters():
    params.requires_grad = True
    seg_params += [params]
for params in model_without_ddp.backbone.feature_selection.parameters():
    params.requires_grad = True
    seg_params += [params]
for params in model_without_ddp.deform_layer.parameters():
    params.requires_grad = True
    seg_params += [params]
for params in model_without_ddp.reg_head.parameters():
    params.requires_grad = True
    seg_params += [params]
for params in model_without_ddp.cls_head.parameters():
    params.requires_grad = True
    seg_params += [params]
for params in model_without_ddp.conv.parameters():
    params.requires_grad = True
    seg_params += [params]
for params in model_without_ddp.mask_head.parameters():
    params.requires_grad = True
    seg_params += [params]

In [26]:
for name, param in model.named_parameters():
    if param.requires_grad:
        # print(f"Parameter '{name}' is trainable.")
        continue
    else:
        print(f"Parameter '{name}' is frozen.")
    # 没输出说明全都解冻

In [7]:
# 打印每一层的参数
for name, param in model.named_parameters():
    # if param.requires_grad:
    if "feature_selection"  in name and  "leakyunit_u_fpn" not in name and "leakyunit_fpn_u" not in name:
      
      print(f"Layer: {name}, Shape: {param.shape}")

In [20]:
for name, param in model.named_parameters():
    # print(f"Layer: {name}, Shape: {param.shape}")
    if name == 'deform_layer.layers.0.weight':
        print(f"Layer: {name}, Shape: {param}")


Layer: deform_layer.layers.0.weight, Shape: Parameter containing:
tensor([[ 0.0391, -0.0611,  0.0527,  ...,  0.0382, -0.0605,  0.0706],
        [ 0.0488,  0.0299, -0.0435,  ..., -0.0055, -0.0340, -0.0035],
        [ 0.0626, -0.0202,  0.0211,  ...,  0.0347, -0.0071, -0.0414],
        ...,
        [-0.0622,  0.0344,  0.0196,  ...,  0.0495,  0.0393,  0.0497],
        [ 0.0242,  0.0107, -0.0018,  ..., -0.0302,  0.0157, -0.0430],
        [-0.0146,  0.0494,  0.0103,  ..., -0.0149,  0.0449, -0.0356]],
       requires_grad=True)


In [3]:
import torch
dir_checkpoint='checkpoint/test1/latest.pth'
checkpoint = torch.load(dir_checkpoint )
print(checkpoint.keys())

# # 提取 state_dict
state_dict = checkpoint['model']  # 如果键是 'state_dict'

# 查看每一层的参数
for name, param in state_dict.items():
    # print(f"Layer: {name}, Shape: {param}")
      # if name == 'deform_layer.layers.0.weight':
        print(f"Layer: {name}, Shape: {param.shape}")

dict_keys(['model', 'optimizer', 'f1', 'epoch', 'args'])
Layer: backbone.backbone.stem.0.weight, Shape: torch.Size([96, 3, 4, 4])
Layer: backbone.backbone.stem.0.bias, Shape: torch.Size([96])
Layer: backbone.backbone.stem.1.weight, Shape: torch.Size([96])
Layer: backbone.backbone.stem.1.bias, Shape: torch.Size([96])
Layer: backbone.backbone.stages.0.blocks.0.gamma, Shape: torch.Size([96])
Layer: backbone.backbone.stages.0.blocks.0.conv_dw.weight, Shape: torch.Size([96, 1, 7, 7])
Layer: backbone.backbone.stages.0.blocks.0.conv_dw.bias, Shape: torch.Size([96])
Layer: backbone.backbone.stages.0.blocks.0.norm.weight, Shape: torch.Size([96])
Layer: backbone.backbone.stages.0.blocks.0.norm.bias, Shape: torch.Size([96])
Layer: backbone.backbone.stages.0.blocks.0.mlp.fc1.weight, Shape: torch.Size([384, 96])
Layer: backbone.backbone.stages.0.blocks.0.mlp.fc1.bias, Shape: torch.Size([384])
Layer: backbone.backbone.stages.0.blocks.0.mlp.fc2.weight, Shape: torch.Size([96, 384])
Layer: backbone.bac

In [11]:
import torch
unet_checkpoint='pretrained/unet_L2_v0.2.2.sav'
ucheckpoint = torch.load(unet_checkpoint )

print(ucheckpoint.keys())

# # 查看每一层的参数
for name, param in ucheckpoint.items():
  # if "backbone.backbone" not in name:
    if name == 'dec5.0.weight':
      print(f"Layer: {name}, Shape: {param}")

odict_keys(['enc1.0.weight', 'enc1.0.bias', 'enc2.0.weight', 'enc2.0.bias', 'enc3.0.weight', 'enc3.0.bias', 'enc4.0.weight', 'enc4.0.bias', 'enc5.0.weight', 'enc5.0.bias', 'enc6.0.weight', 'enc6.0.bias', 'dec5.0.weight', 'dec5.0.bias', 'dec5.2.weight', 'dec5.2.bias', 'dec4.0.weight', 'dec4.0.bias', 'dec4.2.weight', 'dec4.2.bias', 'dec3.0.weight', 'dec3.0.bias', 'dec3.2.weight', 'dec3.2.bias', 'dec2.0.weight', 'dec2.0.bias', 'dec2.2.weight', 'dec2.2.bias', 'dec1.0.weight', 'dec1.0.bias', 'dec1.2.weight', 'dec1.2.bias', 'dec1.4.weight', 'dec1.4.bias'])
Layer: dec5.0.weight, Shape: tensor([[[[ 1.0222e-02,  2.9191e-02,  2.4678e-02],
          [ 2.2373e-02, -1.3180e-02,  1.9336e-02],
          [-2.1471e-02,  1.3956e-03,  2.4064e-02]],

         [[ 2.6783e-03,  3.4265e-02,  2.3746e-02],
          [ 4.2873e-02, -1.1728e-02, -1.5978e-02],
          [ 2.5280e-04, -2.8526e-03, -1.7375e-02]],

         [[-1.4609e-02, -1.2369e-02,  1.3339e-02],
          [-3.1278e-02,  1.2187e-02, -1.6942e-02],
  